In [1]:
'''
Access molecule geometry.

Mole.natm is the total number of atoms.  It is initialized in Mole.build()
function.
'''

from pyscf import gto

mol = gto.M(
    atom = '''
O        0.000000    0.000000    0.117790
H        0.000000    0.755453   -0.471161
H        0.000000   -0.755453   -0.471161''',
    basis = 'ccpvdz',
)


In [2]:
for i in range(mol.natm):
    print('%s %s  charge %f  xyz %s' % (mol.atom_symbol(i),
                                        mol.atom_pure_symbol(i),
                                        mol.atom_charge(i),
                                        mol.atom_coord(i)))

print("Atomic charges:")
print(mol.atom_charges())
print("Atomic coordinates (Bohr):")
print(mol.atom_coords())
print("Atomic coordinates (Angstrom):")
print(mol.atom_coords(unit='Ang'))


O O  charge 8.000000  xyz [0.         0.         0.22259084]
H H  charge 1.000000  xyz [ 0.          1.42759927 -0.89036525]
H H  charge 1.000000  xyz [ 0.         -1.42759927 -0.89036525]
Atomic charges:
[8 1 1]
Atomic coordinates (Bohr):
[[ 0.          0.          0.22259084]
 [ 0.          1.42759927 -0.89036525]
 [ 0.         -1.42759927 -0.89036525]]
Atomic coordinates (Angstrom):
[[ 0.        0.        0.11779 ]
 [ 0.        0.755453 -0.471161]
 [ 0.       -0.755453 -0.471161]]


In [3]:
from pyscf import gto

'''
Access basis information

Mole.nbas is the total number of GTO shells.  It is initialized in
Mole.build() function.

.ao_labels functions can be used to label AO functions.  It has two
formats: tuple or string.  The tuple format has four elements:
    (atom-id, atom-symbol, angular-momentum, xyz-components)
'''

mol = gto.M(
    atom = '''
O        0.000000    0.000000    0.117790
H1       0.000000    0.755453   -0.471161
H2       0.000000   -0.755453   -0.471161''',
    basis = 'ccpvdz',
)

In [4]:
print('total number of shells %d, total number of AO functions %d' %
      (mol.nbas, mol.nao_nr()))

# Filter AO functions using AO labels, in tuple
for label in mol.ao_labels(None):
    if label[2] == '2p' and label[3] == 'z':
        print(label)


total number of shells 11, total number of AO functions 24
(np.int32(0), 'O', '2p', 'z')
(np.int32(1), 'H1', '2p', 'z')
(np.int32(2), 'H2', '2p', 'z')


In [5]:
# Filter AO functions using formated AO labels
for label in mol.ao_labels():
    if '2pz' in label:
        print(label)

for i in range(mol.nbas):
    print('shell %d on atom %d l = %s has %d contracted GTOs' %
          (i, mol.bas_atom(i), mol.bas_angular(i), mol.bas_nctr(i)))


0 O 2pz   
1 H1 2pz   
2 H2 2pz   
shell 0 on atom 0 l = 0 has 2 contracted GTOs
shell 1 on atom 0 l = 0 has 1 contracted GTOs
shell 2 on atom 0 l = 1 has 1 contracted GTOs
shell 3 on atom 0 l = 1 has 1 contracted GTOs
shell 4 on atom 0 l = 2 has 1 contracted GTOs
shell 5 on atom 1 l = 0 has 1 contracted GTOs
shell 6 on atom 1 l = 0 has 1 contracted GTOs
shell 7 on atom 1 l = 1 has 1 contracted GTOs
shell 8 on atom 2 l = 0 has 1 contracted GTOs
shell 9 on atom 2 l = 0 has 1 contracted GTOs
shell 10 on atom 2 l = 1 has 1 contracted GTOs


In [6]:
# mol.search_ao_label is a short-cut function that returns the indices of AOs
# wrt the given AO label pattern
print('\nAOs that contains oxygen p orbitals and hydrogen s orbitals')
ao_labels = mol.ao_labels()
idx = mol.search_ao_label(['O.*p', 'H.*s'])
for i in idx:
    print(i, ao_labels[i])



AOs that contains oxygen p orbitals and hydrogen s orbitals
3 0 O 2px   
4 0 O 2py   
5 0 O 2pz   
6 0 O 3px   
7 0 O 3py   
8 0 O 3pz   
14 1 H1 1s    
15 1 H1 2s    
19 2 H2 1s    
20 2 H2 2s    


In [7]:
import pickle
from pyscf import gto

'''
Serialize Mole object

Mole.pack function transform the Mole object to a Python dict.  It can be
serialized using the standard serialization module in python, eg pickle.

There is a hacky way to serialize the Mole object for broadcasting or saving.
Use format or str function to convert the Mole-python-dict to a string.
Later, use the built-in eval function and Mole.unpack function to restore the
Mole object.
'''

mol = gto.M(
    atom = '''
O        0.000000    0.000000    0.117790
H1       0.000000    0.755453   -0.471161
H2       0.000000   -0.755453   -0.471161''',
    basis = 'ccpvdz',
)

In [8]:
# In Python pickled format
ar = pickle.dumps(format(mol.pack()))
mol1 = gto.unpack(eval(pickle.loads(ar)))

# In JSON format
ar = mol.dumps()
mol1 = gto.loads(ar)



In [11]:
from pyscf import gto

'''
Specify symmetry.

Mole.symmetry can be True/False to turn on/off the symmetry (default is off),
or a string to specify the symmetry of molecule.  symmetry_subgroup keyword
can be set to generate a subgroup of the detected symmetry.
symmetry_subgroup has no effect when an explicit label is assigned to
Mole.symmetry.

Symmetry adapted basis are stored in Mole attribute symm_orb.
'''

mol = gto.M(
    atom = 'C 0 .2 0; O 0 0 1.1',
    symmetry = True,
)
print('Symmetry %-4s, subgroup %s.' % (mol.topgroup, mol.groupname))
print('--\n')


Symmetry Coov, subgroup Coov.
--



In [12]:
print('Symmetry %-4s, subgroup %s.' % (mol.topgroup, mol.groupname))
print('--\n')

mol = gto.M(
    atom = 'C 0 .2 0; O 0 0 1.1',
    symmetry = True,
    symmetry_subgroup = 'C2v',
)
print('Symmetry %-4s, subgroup %s.' % (mol.topgroup, mol.groupname))
print('--\n')

mol = gto.M(
    atom = 'C 0 0 0; O 0 0 1.5',
    symmetry = 'C2v',
)
print('Symmetry %-4s, subgroup %s.' % (mol.topgroup, mol.groupname))
print('If "symmetry=xxx" is specified, the symmetry for the molecule will be set to xxx')
print('--\n')

print('Symmetry adapted orbitals')
for k, ir in enumerate(mol.irrep_name):
    print('Irrep name %s  (ID %d), symm-adapted-basis shape %s' %
          (ir, mol.irrep_id[k], mol.symm_orb[k].shape))


Symmetry Coov, subgroup Coov.
--

Symmetry Coov, subgroup C2v.
--

Symmetry Coov, subgroup C2v.
If "symmetry=xxx" is specified, the symmetry for the molecule will be set to xxx
--

Symmetry adapted orbitals
Irrep name A1  (ID 0), symm-adapted-basis shape (10, 6)
Irrep name B1  (ID 2), symm-adapted-basis shape (10, 2)
Irrep name B2  (ID 3), symm-adapted-basis shape (10, 2)
